### below load dependency pkgs. and then create manually a new directory under experiments like base_model having params.json. this json file contains hyperparameter related to model, user can tweak these parameters for finding best model. and based on new name , change below model_dir path.

In [1]:
import argparse
import logging
import os

import numpy as np
import torch
from torch.autograd import Variable
import torch.optim as optim
from tqdm import trange
import random
from seqeval.metrics import f1_score, accuracy_score, classification_report

# dependancy packages.
import utils
import model.net_w2v_pos as net
from model.data_loader import DataLoader
from model.net_w2v_pos import flat_accc


data_dir='data/small'
model_dir='experiments/base_model2'
word2vec_dir='experiments/word2vec' # keep GoogleNews-vectors-negative300.bin inside this directory.

# ## Let's use CoNLL 2002 data to build a NER system
# 
# CoNLL2002 corpus is available in NLTK. We use Spanish data.

In [11]:
import nltk
from sklearn.model_selection import cross_val_predict
nltk.corpus.conll2002.fileids()

train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
train_sents = [val for val in train_sents if len(val)!=0]
train_sents = train_sents[:5000]
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
test_sents = [val for val in test_sents if len(val)!=0]
test_sents = test_sents[:1000]

print(len(train_sents))
print(len(test_sents))


5000
1000


# convert sentences to list of tokens, pos , labels

In [25]:
tokens_word2vec_list = []
tr_tokens_list = []
tr_pos_list = []
tr_labels_list = []
ts_tokens_list = []
ts_pos_list = []
ts_labels_list = []

def preprocess_data(x,tokens_word2vec_list):
    a = []
    b = []
    c = []
    for sents in x:
        tokens = tuple(i[0].lower() for i in sents)
        pos_tags = [i[1] for i in sents]
        labels = tuple(i[2] for i in sents)
        tokens_word2vec_list.append(tokens)
        a.append(tokens)
        b.append(pos_tags)
        c.append(labels)
    return tokens_word2vec_list,a,b,c

tokens_word2vec_list,tr_tokens_list,tr_pos_list,tr_labels_list = preprocess_data(train_sents,tokens_word2vec_list)
tokens_word2vec_list,ts_tokens_list,ts_pos_list,ts_labels_list = preprocess_data(test_sents,tokens_word2vec_list)
#print(tr_tokens_list[0])
#print(tr_pos_list[:4])
#print(tr_labels_list[0])

# load word_embeddings

In [13]:
from gensim.models import Word2Vec
sentences = tokens_word2vec_list#[["cat", "say", "meow"], ["dog", "say", "woof"]]

emb_model = Word2Vec(min_count=1)
emb_model.build_vocab(sentences)  # prepare the model vocabulary
emb_model.train(sentences, total_examples=emb_model.corpus_count, epochs=emb_model.iter)  # train word vectors

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
collected 19950 word types from a corpus of 192004 raw words and 6000 sentences
Loading a fresh vocabulary
effective_min_count=1 retains 19950 unique words (100% of original 19950, drops 0)
effective_min_count=1 leaves 192004 word corpus (100% of original 192004, drops 0)
deleting the raw counts dictionary of 19950 items
sample=0.001 downsamples 32 most-common words
downsampling leaves estimated 127370 word corpus (66.3% of prior 192004)
estimated required memory for 19950 words and 100 dimensions: 25935000 bytes
resetting layer weights
/home/adzuser/user_achyuta/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  
training model with 3 workers on 19950 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
worker thread finished; awaiting fin

(637027, 960020)

In [14]:
#emb_model['australia'].astype(np.float64)

# Build vocabularies of words and tags from datasets.
# why ? bcs here neural networks input demands vocabulary size and vocabularies.

In [15]:
"""Build vocabularies of words and tags from datasets"""

import argparse
from collections import Counter
import json
import os


parser = {'min_count_word':1,'min_count_tag':1,'data_dir':'data/small'}#argparse.ArgumentParser()
#parser.add_argument('--min_count_word', default=1, help="Minimum count for words in the dataset", type=int)
#parser.add_argument('--min_count_tag', default=1, help="Minimum count for tags in the dataset", type=int)
#parser.add_argument('--data_dir', default='data/small', help="Directory containing the dataset")

# Hyper parameters for the vocab
PAD_WORD = '<pad>'
PAD_TAG = 'O'
UNK_WORD = 'UNK'


def save_vocab_to_txt_file(vocab, txt_path):
    """Writes one token per line, 0-based line id corresponds to the id of the token.

    Args:
        vocab: (iterable object) yields token
        txt_path: (stirng) path to vocab file
    """
    with open(txt_path, "w") as f:
        for token in vocab:
            f.write(token + '\n')
            

def save_dict_to_json(d, json_path):
    """Saves dict to json file

    Args:
        d: (dict)
        json_path: (string) path to json file
    """
    with open(json_path, 'w') as f:
        d = {k: v for k, v in d.items()}
        json.dump(d, f, indent=4)


def update_vocab(tokens_list, vocab):
    """Update word and tag vocabulary from dataset

    Args:
        txt_path: (string) path to file, one sentence per line
        vocab: (dict or Counter) with update method

    Returns:
        dataset_size: (int) number of elements in the dataset
    """
    for i, tokens in enumerate(tokens_list):
        vocab.update(tokens)

    return i + 1


if __name__ == '__main__':
    #args = parser.parse_args()

    # Build word vocab with train and test datasets
    print("Building word vocabulary...")
    words = Counter()
    size_train_sentences = update_vocab(tr_tokens_list, words)
    size_dev_sentences = update_vocab(ts_tokens_list, words)
    size_test_sentences = update_vocab(ts_tokens_list, words)
    print("- done.")

    # Build tag vocab with train and test datasets
    print("Building tag vocabulary...")
    tags = Counter()
    size_train_tags = update_vocab(tr_labels_list, tags)
    size_dev_tags = update_vocab(ts_labels_list, tags)
    size_test_tags = update_vocab(ts_labels_list, tags)
    print("- done.")

    # Assert same number of examples in datasets
    assert size_train_sentences == size_train_tags
    assert size_dev_sentences == size_dev_tags
    assert size_test_sentences == size_test_tags

    # Only keep most frequent tokens
    words = [tok for tok, count in words.items() if count >= parser['min_count_word']]
    tags = [tok for tok, count in tags.items() if count >= parser['min_count_tag']]

    # Add pad tokens
    if PAD_WORD not in words: words.append(PAD_WORD)
    if PAD_TAG not in tags: tags.append(PAD_TAG)
    
    # add word for unknown words 
    words.append(UNK_WORD)

    # Save vocabularies to file
    print("Saving vocabularies to file...")
    save_vocab_to_txt_file(words, os.path.join(parser['data_dir'], 'words.txt'))
    save_vocab_to_txt_file(tags, os.path.join(parser['data_dir'], 'tags.txt'))
    print("- done.")

    # Save datasets properties in json file
    sizes = {
        'train_size': size_train_sentences,
        'dev_size': size_dev_sentences,
        'test_size': size_test_sentences,
        'vocab_size': len(words),
        'number_of_tags': len(tags),
        'pad_word': PAD_WORD,
        'pad_tag': PAD_TAG,
        'unk_word': UNK_WORD
    }
    
    save_dict_to_json(sizes, os.path.join(parser['data_dir'], 'dataset_params.json'))

    # Logging sizes
    to_print = "\n".join("- {}: {}".format(k, v) for k, v in sizes.items())
    print("Characteristics of the dataset:\n{}".format(to_print))

Building word vocabulary...
- done.
Building tag vocabulary...
- done.
Saving vocabularies to file...
- done.
Characteristics of the dataset:
- train_size: 5000
- dev_size: 1000
- test_size: 1000
- vocab_size: 19952
- number_of_tags: 9
- pad_word: <pad>
- pad_tag: O
- unk_word: UNK


# convert pos-tags to onehot vectors

In [26]:
from numpy import argmax
import itertools

# define input string
data = ['NP', 'Fpa', 'NP', 'Fpt', 'Fc', 'Z', 'NC', 'Fpa', 'NC', 'Fpt', 'Fp']

# define universe of possible input values
unq_pos_list = list(itertools.chain.from_iterable(tr_pos_list+ts_pos_list))
unq_pos_list = list(set(unq_pos_list))

# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(unq_pos_list))
int_to_char = dict((i, c) for i, c in enumerate(unq_pos_list))

def onehot_encode(data,unq_pos_list,char_to_int):
    # integer encode input data
    integer_encoded = [char_to_int[pos] for pos in data]

    # one hot encode
    onehot_encoded = list()
    for value in integer_encoded:
        letter = [0 for _ in range(len(unq_pos_list))]
        letter[value] = 1
        onehot_encoded.append(letter)
    return onehot_encoded

onehot_encoded = onehot_encode(data,unq_pos_list,char_to_int)

# invert encoding
#inverted = int_to_char[argmax(onehot_encoded[0])]
#print(inverted)

# train & evaluation functions.

In [27]:
class data_loader(object):
    def data_iterator(self, data_dir, data, params, shuffle=False):
        """
        Returns a generator that yields batches data with labels. Batch size is params.batch_size. Expires after one
        pass over the data.

        Args:
            data: (dict) contains data which has keys 'data', 'labels' and 'size'
            params: (Params) hyperparameters of the training process.
            shuffle: (bool) whether the data should be shuffled

        Yields:
            batch_data: (Variable) dimension batch_size x seq_len with the sentence data
            batch_labels: (Variable) dimension batch_size x seq_len with the corresponding labels

        """

        # make a list that decides the order in which we go over the data- this avoids explicit shuffling of data
        order = list(range(data['size']))
        if shuffle:
            random.seed(230)
            random.shuffle(order)

        # loading dataset_params
        json_path = os.path.join(data_dir, 'dataset_params.json')
        assert os.path.isfile(json_path), "No json file found at {}, run build_vocab.py".format(json_path)
        dataset_params = utils.Params(json_path)        

        # loading vocab (we require this to map words to their indices)
        vocab_path = os.path.join(data_dir, 'words.txt')
        vocab = {}
        with open(vocab_path) as f:
            for i, l in enumerate(f.read().splitlines()):
                vocab[l] = i

        # setting the indices for UNKnown words and PADding symbols
        self.unk_ind = vocab[dataset_params.unk_word]
        self.pad_ind = vocab[dataset_params.pad_word]
        
        tr_tokens_list_ind = []
        for sent_token_list in data['data']:
            s = [vocab[token] if token in vocab else self.unk_ind for token in sent_token_list]
            tr_tokens_list_ind.append(s)
        data['data'] = tr_tokens_list_ind
        

        # one pass over data
        for i in range((data['size']+1)//params.batch_size):
            # fetch sentences and tags
            batch_sentences = [data['data'][idx] for idx in order[i*params.batch_size:(i+1)*params.batch_size]]
            batch_tags = [data['labels'][idx] for idx in order[i*params.batch_size:(i+1)*params.batch_size]]
            batch_pos = [data['pos'][idx] for idx in order[i*params.batch_size:(i+1)*params.batch_size]]
            
            # compute length of longest sentence in batch
            batch_max_len = max([len(s) for s in batch_sentences])

            # prepare a numpy array with the data, initialising the data with pad_ind and all labels with -1
            # initialising labels to -1 differentiates tokens with tags from PADding tokens
            batch_data = self.pad_ind*np.ones((len(batch_sentences), batch_max_len))
            batch_labels = -1*np.ones((len(batch_sentences), batch_max_len))
            batch_pos_w = np.zeros((len(batch_sentences), batch_max_len, params.pos_dim))
            
            # copy the data to the numpy array
            for j in range(len(batch_sentences)):
                cur_len = len(batch_sentences[j])
                batch_data[j][:cur_len] = batch_sentences[j]
                batch_labels[j][:cur_len] = batch_tags[j]
                batch_pos_w[j][:cur_len] = batch_pos[j]

            # since all data are indices, we convert them to torch LongTensors
            batch_data, batch_labels, batch_pos_w = torch.LongTensor(batch_data), \
            torch.LongTensor(batch_labels), torch.FloatTensor(batch_pos_w)
            
            # shift tensors to GPU if available
            if params.cuda:
                batch_data, batch_labels, batch_pos_w = batch_data.cuda(), batch_labels.cuda(), batch_pos_w.cuda()

            # convert them to Variables to record operations in the computational graph
            batch_data, batch_labels, batch_pos_w = Variable(batch_data), Variable(batch_labels), Variable(batch_pos_w)
            
            yield batch_data, batch_labels, batch_pos_w, batch_tags


def train(model, optimizer, loss_fn, data_iterator, metrics, params, num_steps,tag_map_rev):
    """Train the model on `num_steps` batches

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_sum = 0.0
    loss_avg = utils.RunningAverage()
    
    # Use tqdm for progress bar
    t = trange(num_steps)
    batch_len = 0
    for i in t:
        # fetch the next training batch
        train_batch, labels_batch, batch_pos_w, batch_tags = next(data_iterator)
        
        # compute model output and loss
        loss = model(train_batch,batch_pos_w,labels_batch)
        output_batch = model.forward_decode(train_batch,batch_pos_w,labels_batch)
        #print(loss)
        #print(output_batch)
        #print("labels_batch : ",labels_batch)
        batch_acc = flat_accc(output_batch, labels_batch)
        #print("batch_acc : ",batch_acc)
        #print("f1_score : ",f1_score(batch_tags,output_batch))
        #print("accuracy_score : ",accuracy_score(batch_tags,output_batch))
        #import pdb;pdb.set_trace()
        #loss = loss_fn(output_batch, labels_batch)
        loss = torch.mean(loss)

        # clear previous gradients, compute gradients of all variables wrt loss
        optimizer.zero_grad()
        loss.backward()

        # performs updates using calculated gradients
        optimizer.step()

        # Evaluate summaries only once in a while
        if i % params.save_summary_steps == 0:
            # extract data from torch Variable, move to cpu, convert to numpy arrays
            #output_batch = output_batch.data.cpu().numpy()
            #labels_batch = labels_batch.data.cpu().numpy()

            # compute all metrics on this batch
            summary_batch = {metric:metrics[metric](output_batch, labels_batch)
                             for metric in metrics}
            summary_batch['loss'] = loss.data.item()
            summ.append(summary_batch)

        loss1 = loss.data.item()
        loss_sum += loss1
        
        # update the average loss
        loss_avg.update(loss.data.item())
        t.set_postfix(loss='{:05.3f}'.format(loss_avg()))
        batch_len += 1

    # compute mean of all metrics in summary
    loss_sum /= batch_len
    print("train loss_sum : ",loss_sum)
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]} 
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    

def evaluate(model, loss_fn, data_iterator, metrics, params, num_steps, tag_map_rev):
    """Evaluate the model on `num_steps` batches.

    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to evaluation mode
    model.eval()

    # summary for current eval loop
    summ = []
    summ_batch_tags = []
    summ_batch_pred = []

    # compute metrics over the dataset
    for _ in range(num_steps):
        # fetch the next evaluation batch
        data_batch, labels_batch, batch_pos_w, batch_tags = next(data_iterator)
        
        summ_batch_tags.extend(batch_tags)
        
        # compute model output
        #output_batch = model.decode(data_batch,batch_pos_w)
        loss = model(data_batch,batch_pos_w,labels_batch)
        #loss = loss_fn(output_batch, labels_batch)
        output_batch = model.forward_decode(data_batch,batch_pos_w,labels_batch)
        
        summ_batch_pred.extend(output_batch)
        #print(loss)
        #print(output_batch)
        batch_acc = flat_accc(output_batch, labels_batch)
        print("batch_acc : ",batch_acc)
        
        #print("f1_score : ",f1_score(batch_tags,output_batch))
        #print("accuracy_score : ",accuracy_score(batch_tags,output_batch))
        #print(loss)
        #print(output_batch)
        #import pdb;pdb.set_trace()

        # extract data from torch Variable, move to cpu, convert to numpy arrays
        #output_batch = output_batch.data.cpu().numpy()
        #labels_batch = labels_batch.data.cpu().numpy()

        # compute all metrics on this batch
        summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                         for metric in metrics}
        summary_batch['loss'] = loss.data.item()
        summ.append(summary_batch)

    # compute mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]} 
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    summ_batch_pred_n = [[tag_map_rev[each_val] for each_val in val] for val in summ_batch_pred]
    summ_batch_tags_n = [[tag_map_rev[each_val] for each_val in val] for val in summ_batch_tags]
    #print("&&&&&&&&& : ",summ_batch_pred_n[:5])
    #print("&&&&&&&&& : ",summ_batch_tags_n[:5])
    print("f1_score : ",f1_score(summ_batch_tags_n,summ_batch_pred_n))
    print("accuracy_score : ",accuracy_score(summ_batch_tags_n,summ_batch_pred_n))
    
    return metrics_mean

def train_and_evaluate(model, train_data, val_data, optimizer, loss_fn, metrics, params, model_dir, 
                       data_dir, tag_map_rev, restore_file=None):
    """Train the model and evaluate every epoch.

    Args:
        model: (torch.nn.Module) the neural network
        train_data: (dict) training data with keys 'data' and 'labels'
        val_data: (dict) validaion data with keys 'data' and 'labels'
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        model_dir: (string) directory containing config, weights and log
        restore_file: (string) optional- name of file to restore from (without its extension .pth.tar)
    """
    # reload weights from restore_file if specified
    if restore_file is not None:
        restore_path = os.path.join(args.model_dir, args.restore_file + '.pth.tar')
        utils.load_checkpoint(restore_path, model, optimizer)
        
    best_val_acc = 0.0

    for epoch in range(params.num_epochs):
        # Run one epoch
        # compute number of batches in one epoch (one full pass over the training set)
        num_steps = (params.train_size + 1) // params.batch_size
        dl = data_loader()
        train_data_iterator = dl.data_iterator(data_dir, train_data, params, shuffle=True)
        train(model, optimizer, loss_fn, train_data_iterator, metrics, params, num_steps,tag_map_rev)
            
        # Evaluate for one epoch on validation set
        num_steps = (params.val_size + 1) // params.batch_size
        val_data_iterator = dl.data_iterator(data_dir, val_data, params, shuffle=False)
        val_metrics = evaluate(model, loss_fn, val_data_iterator, metrics, params, num_steps,tag_map_rev)
        print("val_metrics : ",val_metrics)
        
        val_acc = val_metrics['accuracy']
        is_best = val_acc >= best_val_acc

        # Save weights
        utils.save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict' : optimizer.state_dict()}, 
                               is_best=True,
                               checkpoint=model_dir)
            
        # If best_eval, best_save_path        
        if is_best:
            best_val_acc = val_acc
            
            # Save best val metrics in a json file in the model directory
            best_json_path = os.path.join(model_dir, "metrics_val_best_weights.json")
            utils.save_dict_to_json(val_metrics, best_json_path)

        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(model_dir, "metrics_val_last_weights.json")
        utils.save_dict_to_json(val_metrics, last_json_path)

# more util functions.

In [32]:
def get_word_emdd_weights():
    # loading vocab (we require this to map words to their indices)
    vocab_path = os.path.join(data_dir, 'words.txt')
    vocab = {}
    with open(vocab_path) as f:
        for i, l in enumerate(f.read().splitlines()):
            vocab[l] = i

    weight = np.zeros((len(vocab),params.embedding_dim))
    vocab_inv = {y:x for x,y in vocab.items()}
    for ind,val in vocab_inv.items():
        try:
            weight[ind][:]= emb_model[val]
        except:
            print("exception word not have embeddings : ",val)
    return weight

def preprocess_postags(pos_list):
    '''tr_data = []
    for v in tokens_list:
        tmp_tr_data = []
        for v_in in v:
            tmp_tr_data.append(emb_model[v_in.lower()].tolist()) #.astype(np.float64)
        tr_data.append(tmp_tr_data)
    '''
    tr_pos = [onehot_encode(v,unq_pos_list,char_to_int) for v in pos_list]
    return tr_pos

def preprocess_label(tag_map, labels_list):
    fnl_labels = []
    for label_sent in labels_list:
        tmp_label = []
        for label in label_sent:
            tmp_label.append(tag_map[label])
        fnl_labels.append(tmp_label)
    return fnl_labels

def get_tags():
    tags_path = os.path.join(data_dir, 'tags.txt')
    tag_map = {}
    with open(tags_path) as f:
        for i, t in enumerate(f.read().splitlines()):
            tag_map[t] = i
    return tag_map,{j:i for i,j in tag_map.items()}

def make_equal(x,y):
    c = []
    d = []
    for i,j in zip(x,y):
        if len(i) != len(j):
            print("not equal length : ")
            continue
        c.append(i)
        d.append(j)
    return c,d

# Train

In [34]:
restore_file=None

# Load the parameters from json file
json_path = os.path.join(model_dir, 'params.json')
assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
params = utils.Params(json_path)

# use GPU if available
params.cuda = torch.cuda.is_available()
    
# Set the random seed for reproducible experiments
torch.manual_seed(230)
if params.cuda: torch.cuda.manual_seed(230)
        
# Set the logger
utils.set_logger(os.path.join(model_dir, 'train.log'))

# Create the input data pipeline
logging.info("Loading the datasets...")

params.vocab_size=sizes['vocab_size']
params.number_of_tags = sizes['number_of_tags']

# convert pos-tags in data to one-hot vector.
tr_pos = None
ts_pos = None
tr_pos = preprocess_postags(tr_pos_list)
ts_pos = preprocess_postags(ts_pos_list)

# loading tags (we require this to map tags to their indices)
tag_map,tag_map_rev = get_tags()

# convert labels/tags to index.
tr_label_list = preprocess_label(tag_map, tr_labels_list)
ts_label_list = preprocess_label(tag_map, ts_labels_list)

# select if tokens and labels size equal.
tr_tokens_list_1, tr_label_list_1 = make_equal(tr_tokens_list,tr_label_list)
ts_tokens_list_1, ts_label_list_1 = make_equal(ts_tokens_list,ts_label_list)

# short it for faster buildings. can comment below section for including all data.
tr_tokens_list_1 = tr_tokens_list_1[:500]
tr_pos = tr_pos[:500]
tr_label_list_1 = tr_label_list_1[:500]
ts_tokens_list_1 = ts_tokens_list_1[:100]
ts_pos = ts_pos[:100]
ts_label_list_1 = ts_label_list_1[:100]

train_data = {'data':tr_tokens_list_1,'pos':tr_pos,'labels':tr_label_list_1,'size':len(tr_tokens_list_1)}
val_data = {'data':ts_tokens_list_1,'pos':ts_pos,'labels':ts_label_list_1,'size':len(ts_tokens_list_1)}

# specify the train and val dataset sizes
params.train_size = train_data['size']
params.val_size = val_data['size']

# get weight matrix for each vocab to initialize model with pre-trained embeddings.
weight = get_word_emdd_weights()

# Define the model and optimizer
model = net.Netcp(params).cuda() if params.cuda else net.Net(params)
model.init_word_embeddings(torch.FloatTensor(weight).cuda())
optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
    
# fetch loss function and metrics
loss_fn = net.loss_fn
metrics = net.metrics

# Train the model
logging.info("Starting training for {} epoch(s)".format(params.num_epochs))
train_and_evaluate(model, train_data, val_data, optimizer, loss_fn, metrics, params, model_dir, data_dir, tag_map_rev,
                    restore_file)

Loading the datasets...
/home/adzuser/user_achyuta/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
Starting training for 10 epoch(s)

  0%|          | 0/10 [00:00<?, ?it/s]

exception word not have embeddings :  <pad>
exception word not have embeddings :  UNK



100%|██████████| 10/10 [00:13<00:00,  1.44s/it, loss=-9580.864]

train loss_sum :  -9580.86376953125
batch_acc :  0.0077083333333333335
batch_acc :  0.0064285714285714285
f1_score :  0.008047044258743424
accuracy_score :  0.018054432767448128
val_metrics :  {'accuracy': 0.007068452380952381, 'loss': -14817.91064453125}
Checkpoint Directory exists! 



100%|██████████| 10/10 [00:07<00:00,  2.37it/s, loss=-8997.605]

...

train loss_sum :  -70390.789453125
batch_acc :  0.009583333333333333
batch_acc :  0.006666666666666667
f1_score :  0.012858232537866405
accuracy_score :  0.02021018593371059
val_metrics :  {'accuracy': 0.008125, 'loss': -94542.0078125}
Checkpoint Directory exists! 



100%|██████████| 10/10 [00:11<00:00,  1.06s/it, loss=-79353.850]

train loss_sum :  -79353.85
batch_acc :  0.011041666666666667
batch_acc :  0.010238095238095239
f1_score :  0.014675731088809756
accuracy_score :  0.026138507140932365
val_metrics :  {'accuracy': 0.010639880952380953, 'loss': -105546.78125}
Checkpoint Directory exists! 



100%|██████████| 10/10 [00:08<00:00,  1.01s/it, loss=-88087.219]

train loss_sum :  -88087.21875
batch_acc :  0.01375
batch_acc :  0.011666666666666667
f1_score :  0.014675731088809756
accuracy_score :  0.03233629749393695
val_metrics :  {'accuracy': 0.012708333333333334, 'loss': -116340.4296875}
Checkpoint Directory exists! 


# Evaluation

In [35]:
"""
    Evaluate the model on the test set.
"""

restore_file='best'

# Load the parameters
json_path = os.path.join(model_dir, 'params.json')
assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
params = utils.Params(json_path)

# use GPU if available
params.cuda = torch.cuda.is_available()     # use GPU is available

# Set the random seed for reproducible experiments
torch.manual_seed(230)
if params.cuda: torch.cuda.manual_seed(230)

params.vocab_size=sizes['vocab_size']
params.number_of_tags = sizes['number_of_tags']

# load data
#data_loader = DataLoader(data_dir, params)
#data = data_loader.load_data(['test'], data_dir)
#test_data = data['test']

test_data = val_data

# specify the test set size
params.test_size = test_data['size']
#test_data_iterator = data_loader.data_iterator(test_data, params)
dll = data_loader()
test_data_iterator = dll.data_iterator(data_dir, test_data, params)

# Define the model
model = net.Netcp(params).cuda() if params.cuda else net.Net(params)
    
loss_fn = net.loss_fn
metrics = net.metrics

# Reload weights from the saved file
utils.load_checkpoint(os.path.join(model_dir, restore_file + '.pth.tar'), model)

# Evaluate
num_steps = (params.test_size + 1) // params.batch_size
test_metrics = evaluate(model, loss_fn, test_data_iterator, metrics, params, num_steps, tag_map_rev)
save_path = os.path.join(model_dir, "metrics_test_{}.json".format(restore_file))
#utils.save_dict_to_json(test_metrics, save_path)
print(test_metrics)

batch_acc :  0.01375
batch_acc :  0.011666666666666667
f1_score :  0.014675731088809756
accuracy_score :  0.03233629749393695
{'accuracy': 0.012708333333333334, 'loss': -116340.4296875}
